# Manage flow cytometry data

In [ ]:
!lamin init --storage ./test-flow --schema bionty

In [ ]:
import lamindb as ln
import lnschema_bionty as lb
import readfcs

ln.settings.verbosity = 3  # show hints
lb.settings.species = "human"  # globally set species

In [ ]:
ln.track()

We start with a flow cytometry file from Alpert19:

In [ ]:
filepath = ln.dev.datasets.file_fcs_alpert19()

In [ ]:
filepath

Use [readfcs](https://lamin.ai/docs/readfcs) to read the fcs file into memory:

In [ ]:
adata = readfcs.read("Alpert19.fcs")
adata

## Track data with cell markers

We'll use the `CellMarker` reference to link features:

In [ ]:
file = ln.File.from_anndata(adata, description="Alpert19", var_ref=lb.CellMarker.name)
file.save()

In [ ]:
file.features

In [ ]:
file.features["var"].df().head(10)

Let's register another flow file:

In [ ]:
adata2 = readfcs.read(ln.dev.datasets.file_fcs())
file2 = ln.File.from_anndata(
    adata2, description="My fcs file", var_ref=lb.CellMarker.name
)
file2.save()

## Query by cell markers

Which datasets have CD14 in the flow panel:

In [ ]:
cell_markers = lb.CellMarker.lookup()

In [ ]:
cell_markers.cd14

In [ ]:
ln.File.filter(feature_sets__cell_markers=cell_markers.cd14).df()

Shared cell markers between two files:

In [ ]:
file1 = ln.File.filter(feature_sets__cell_markers=cell_markers.cd14).all()[0]
file2 = ln.File.filter(feature_sets__cell_markers=cell_markers.cd14).all()[1]

In [ ]:
file1_markers = file1.features["var"]
file2_markers = file2.features["var"]

shared_markers = file1_markers & file2_markers
shared_markers.list("name")

## Flow marker registry

Check out the first 10 rows of the CellMarker table:

In [ ]:
lb.CellMarker.filter().df().head(10)

In [ ]:
# throw error if there is none
ln.File.filter(feature_sets__cell_markers=cell_markers.cd14).exists()
# clean up test instance
!lamin delete test-flow
!rm -r test-flow